## Performance Profiling with `perf`

### Description 

A  _very_ simple introduction to performance profiling with `perf`. Also, covers the following

   * hardware performance counters and performance events  
   * relative performance and normalized performance   
   * Tools: `hwloc`  
   * Linux utilities: `awk`, `sed`  

### Outline 

  * [Environment Set-up](#env)
  * [Performance Measurement](#perf_measure)
  * [`perf` Basics](#perf)
  * [Normalized Performance](#norm_perf)

### <a name="env"></a>Environment Set-up
   

##### (i) Login to remote linux system

In [ ]:
ssh knuth.cs.txstate.edu 

Set up a working directory for the experiments.

In [1]:
cd
mkdir perf_experiments 
cd perf_experiments 

##### (ii) Get familiar with the experimental environment

Get information about the OS and architecture.

In [2]:
uname -a

Linux minksy.cs.txstate.edu 5.4.0-77-generic #86~18.04.1-Ubuntu SMP Fri Jun 18 01:23:22 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


See who else is logged on and what they are doing

In [3]:
w

 15:06:40 up 46 days, 23:53,  2 users,  load average: 0.04, 0.03, 0.00
USER     TTY      FROM             LOGIN@   IDLE   JCPU   PCPU WHAT
aq10     pts/0    147.26.156.10    14:53   10:29   4.40s  0.02s sshd: aq10 [pri
aq10     pts/1    147.26.156.10    14:57    1:12   0.06s  0.06s -bash


Get CPU information

In [ ]:
cat /proc/cpuinfo

Get memory information

In [ ]:
cat /proc/meminfo

The `hwloc` software package provides command-line tools and a C API to probe the system and get
a more detailed information of compute and memory resources. `hwloc` is usually not pre-installed. It
distributed with BSD licence and can be obtained from the [OpenMPI project
website](https://www.open-mpi.org/projects/hwloc/). `hwloc` has several command-line tools, the most
basic will give a hierarchical map of the compute elements and memory units.

In [7]:
hwloc-ls

Machine (31GB)
  Package L#0 + L3 L#0 (20MB)
    L2 L#0 (256KB) + L1d L#0 (32KB) + L1i L#0 (32KB) + Core L#0
      PU L#0 (P#0)
      PU L#1 (P#8)
    L2 L#1 (256KB) + L1d L#1 (32KB) + L1i L#1 (32KB) + Core L#1
      PU L#2 (P#1)
      PU L#3 (P#9)
    L2 L#2 (256KB) + L1d L#2 (32KB) + L1i L#2 (32KB) + Core L#2
      PU L#4 (P#2)
      PU L#5 (P#10)
    L2 L#3 (256KB) + L1d L#3 (32KB) + L1i L#3 (32KB) + Core L#3
      PU L#6 (P#3)
      PU L#7 (P#11)
    L2 L#4 (256KB) + L1d L#4 (32KB) + L1i L#4 (32KB) + Core L#4
      PU L#8 (P#4)
      PU L#9 (P#12)
    L2 L#5 (256KB) + L1d L#5 (32KB) + L1i L#5 (32KB) + Core L#5
      PU L#10 (P#5)
      PU L#11 (P#13)
    L2 L#6 (256KB) + L1d L#6 (32KB) + L1i L#6 (32KB) + Core L#6
      PU L#12 (P#6)
      PU L#13 (P#14)
    L2 L#7 (256KB) + L1d L#7 (32KB) + L1i L#7 (32KB) + Core L#7
      PU L#14 (P#7)
      PU L#15 (P#15)
  HostBridge L#0
    PCIBridge
      PCI 1000:0097
        Block(Disk) L#0 "sdb"
        Block(Disk) L#1 "sda"
    PCIBridge
  

##### (iii) Obtain code samples

Clone the course git repo on this server.

In [9]:
git clone https://git.txstate.edu/aq10/CS3339.git ~/CS3339.git

Username for 'https://git.txstate.edu': 


Copy the matrix-vector multiplication source file (`matvec.c`) to your working directory

In [11]:
cp ~/CS3339.git/code_samples/matvec.c .

##### (iv) Build and execute

Build the code.

In [12]:
g++ -o matvec matvec.c

Run the executable

In [13]:
./matvec 2000 200

522738635.7446


### <a name="perf_measure"></a>Performance Measurement

**How do we measure the performance of a program?** 

We can use the `time` command to get a rough measure of the execution time. The terms _execution
time_ and _running time_ are synonymous. Runtime means something different!

In [14]:
time ./matvec 2000 200

522738635.7446

real	0m4.167s
user	0m4.155s
sys	0m0.012s


The `time` command reports three numbers. `real` time is the time that has elapsed during the
execution of the program. `user` time is the actual time the program is running on the
processor. `sys` is the time when the _system_ is doing some work either on behalf of this program
or some program. Often `real` time is roughly equal to `user` time + `sys` time 

**Are we happy with this performance of matvec?**

#### Relative performance 

Login to another remote Linux system and create a working directory.

In [ ]:
ssh capi.cs.txstate.edu 
mkdir perf_experiments 
cd perf_experiments 

Check out the environment.

In [ ]:
uname -a 
cat /proc/cpuinfo 
hwloc-ls 
g++ --version 

Clone the course git repo on this server and copy the matrix-vector multiplication code to the working directory.

In [ ]:
git clone https://git.txstate.edu/aq10/CS3339.git ~/CS3339.git
cp ~/CS3339.git/code_samples/matvec.c .

Build and run the `matvec` code with the same arguments and record the execution time.

In [ ]:
g++ -o matvec matvec.c
time ./matvec 2000 200

**Which system is doing better? Do the results match your expectation?** 

Minimum execution time does not necessarily imply the best performance! There are many factors to
consider. 

### <a name="perf"></a>`perf` Basics

Let's go back to our first machine. 

We can check if `perf` is installed just by typing the `perf` command.

In [15]:
perf


 usage: perf [--version] [--help] [OPTIONS] COMMAND [ARGS]

 The most commonly used perf commands are:
   annotate        Read perf.data (created by perf record) and display annotated code
   archive         Create archive with object files with build-ids found in perf.data file
   bench           General framework for benchmark suites
   buildid-cache   Manage build-id cache.
   buildid-list    List the buildids in a perf.data file
   c2c             Shared Data C2C/HITM Analyzer.
   config          Get and set variables in a configuration file.
   data            Data file related processing
   diff            Read perf.data files and display the differential profile
   evlist          List the event names in a perf.data file
   ftrace          simple wrapper for kernel's ftrace functionality
   inject          Filter to augment the events stream with additional information
   kallsyms        Searches running kernel for symbols
   kmem            Tool to trace/measure kernel memory 

: 1

Recent versions of Ubuntu is likely to have `perf` pre-installed. If `perf` is not installed we can
install it with the following.

In [ ]:
sudo apt-get install linux-tools-common linux-tools-generic linux-tools-`uname -r`

Notice the use of back ticks in the above command. You need the `linux-tools-*` package that matches
your kernel. 

Get a basic profile of the `ls` command.

In [16]:
perf stat ls

matvec	matvec.c

 Performance counter stats for 'ls':

              1.39 msec task-clock                #    0.063 CPUs utilized          
                 0      context-switches          #    0.000 K/sec                  
                 0      cpu-migrations            #    0.000 K/sec                  
                92      page-faults               #    0.066 M/sec                  
         1,913,340      cycles                    #    1.380 GHz                    
         1,758,159      instructions              #    0.92  insn per cycle         
           342,822      branches                  #  247.300 M/sec                  
            12,827      branch-misses             #    3.74% of all branches        

       0.022107278 seconds time elapsed

       0.001874000 seconds user
       0.000000000 seconds sys




Get a basic profile of the matrix-vector multiplication code

In [17]:
perf stat ./matvec 2000 200

522738635.7446

 Performance counter stats for './matvec 2000 200':

          4,146.53 msec task-clock                #    1.000 CPUs utilized          
                 4      context-switches          #    0.001 K/sec                  
                 0      cpu-migrations            #    0.000 K/sec                  
             7,882      page-faults               #    0.002 M/sec                  
     7,438,536,419      cycles                    #    1.794 GHz                    
    17,148,686,606      instructions              #    2.31  insn per cycle         
     1,683,384,671      branches                  #  405.974 M/sec                  
           590,791      branch-misses             #    0.04% of all branches        

       4.146975842 seconds time elapsed

       4.130880000 seconds user
       0.016011000 seconds sys




**Do we have any new insight about the performance of matvec?**

The set of performance metrics reported with `perf stat` are not the only ones we can get from
`perf`. The `perf` tool allows us to measure a measure a large number of program _events_. To find
the list of available performance events we can use the `perf list` command.

In [18]:
perf list

  branch-instructions OR branches                    [Hardware event]
  branch-misses                                      [Hardware event]
  bus-cycles                                         [Hardware event]
  cache-misses                                       [Hardware event]
  cache-references                                   [Hardware event]
  cpu-cycles OR cycles                               [Hardware event]
  instructions                                       [Hardware event]
  ref-cycles                                         [Hardware event]
  alignment-faults                                   [Software event]
  bpf-output                                         [Software event]
  context-switches OR cs                             [Software event]
  cpu-clock                                          [Software event]
  cpu-migrations OR migrations                       [Software event]
  dummy                                              [Software event]
  emulation-faults  

  l2_rqsts.all_rfo                                  
       [RFO requests to L2 cache]
  l2_rqsts.code_rd_hit                              
       [L2 cache hits when fetching instructions, code reads]
  l2_rqsts.code_rd_miss                             
       [L2 cache misses when fetching instructions]
  l2_rqsts.demand_data_rd_hit                       
       [Demand Data Read requests that hit L2 cache Spec update: HSD78]
  l2_rqsts.demand_data_rd_miss                      
       [Demand Data Read miss L2, no rejects Spec update: HSD78]
  l2_rqsts.l2_pf_hit                                
       [L2 prefetch requests that hit L2 cache]
  l2_rqsts.l2_pf_miss                               
       [L2 prefetch requests that miss L2 cache]
  l2_rqsts.miss                                     
       [All requests that miss L2 cache Spec update: HSD78]
  l2_rqsts.references                               
       [All L2 requests Spec update: HSD78]
  l2_rqsts.rfo_hit                   

        HSD61]
  offcore_requests_outstanding.cycles_with_demand_rfo
       [Offcore outstanding demand rfo reads transactions in SuperQueue (SQ),
        queue to uncore, every cycle Spec update: HSD62, HSD61]
  offcore_requests_outstanding.demand_code_rd       
       [Offcore outstanding code reads transactions in SuperQueue (SQ), queue
        to uncore, every cycle Spec update: HSD62, HSD61]
  offcore_requests_outstanding.demand_data_rd       
       [Offcore outstanding Demand Data Read transactions in uncore queue Spec
        update: HSD78, HSD62, HSD61]
  offcore_requests_outstanding.demand_data_rd_ge_6  
       [Cycles with at least 6 offcore outstanding Demand Data Read
        transactions in uncore queue Spec update: HSD78, HSD62, HSD61]
  offcore_requests_outstanding.demand_rfo           
       [Offcore outstanding RFO store transactions in SuperQueue (SQ), queue
        to uncore Spec update: HSD62, HSD61]
  offcore_response                                  
       [Off

  idq.mite_cycles                                   
       [Cycles when uops are being delivered to Instruction Decode Queue (IDQ)
        from MITE path]
  idq.mite_uops                                     
       [Uops delivered to Instruction Decode Queue (IDQ) from MITE path]
  idq.ms_cycles                                     
       [Cycles when uops are being delivered to Instruction Decode Queue (IDQ)
        while Microcode Sequenser (MS) is busy]
  idq.ms_dsb_cycles                                 
       [Cycles when uops initiated by Decode Stream Buffer (DSB) are being
        delivered to Instruction Decode Queue (IDQ) while Microcode Sequenser
        (MS) is busy]
  idq.ms_dsb_occur                                  
       [Deliveries to Instruction Decode Queue (IDQ) initiated by Decode
        Stream Buffer (DSB) while Microcode Sequenser (MS) is busy]
  idq.ms_dsb_uops                                   
       [Uops initiated by Decode Stream Buffer (DSB) that are b

       [Counts all demand code reads miss the L3 and the data is returned from
        local dram]
  offcore_response.demand_data_rd.llc_miss.any_response
       [Counts demand data reads miss in the L3]
  offcore_response.demand_data_rd.llc_miss.local_dram
       [Counts demand data reads miss the L3 and the data is returned from
        local dram]
  offcore_response.demand_rfo.llc_miss.any_response 
       [Counts all demand data writes (RFOs) miss in the L3]
  offcore_response.demand_rfo.llc_miss.local_dram   
       [Counts all demand data writes (RFOs) miss the L3 and the data is
        returned from local dram]
  offcore_response.demand_rfo.llc_miss.remote_hitm  
       [Counts all demand data writes (RFOs) miss the L3 and the modified data
        is transferred from remote cache]
  offcore_response.pf_l2_code_rd.llc_miss.any_response
       [Counts all prefetch (that bring data to LLC only) code reads miss in
        the L3]
  offcore_response.pf_l2_data_rd.llc_miss.any_respo

...

  xfs:xfs_buf_submit                                 [Tracepoint event]
  xfs:xfs_buf_trylock                                [Tracepoint event]
  xfs:xfs_buf_trylock_fail                           [Tracepoint event]
  xfs:xfs_buf_unlock                                 [Tracepoint event]
  xfs:xfs_buf_wait_buftarg                           [Tracepoint event]
  xfs:xfs_bunmap                                     [Tracepoint event]
  xfs:xfs_collapse_file_space                        [Tracepoint event]
  xfs:xfs_create                                     [Tracepoint event]
  xfs:xfs_da_fixhashpath                             [Tracepoint event]
  xfs:xfs_da_grow_inode                              [Tracepoint event]
  xfs:xfs_da_join                                    [Tracepoint event]
  xfs:xfs_da_link_after                              [Tracepoint event]
  xfs:xfs_da_link_before                             [Tracepoint event]
  xfs:xfs_da_node_add                                [Tracepoint

  xfs:xfs_file_direct_write                          [Tracepoint event]
  xfs:xfs_file_fsync                                 [Tracepoint event]
  xfs:xfs_file_ioctl                                 [Tracepoint event]
  xfs:xfs_filemap_fault                              [Tracepoint event]
  xfs:xfs_filestream_free                            [Tracepoint event]
  xfs:xfs_filestream_lookup                          [Tracepoint event]
  xfs:xfs_filestream_pick                            [Tracepoint event]
  xfs:xfs_filestream_scan                            [Tracepoint event]
  xfs:xfs_free_extent                                [Tracepoint event]
  xfs:xfs_free_file_space                            [Tracepoint event]
  xfs:xfs_fs_mark_healthy                            [Tracepoint event]
  xfs:xfs_fs_mark_sick                               [Tracepoint event]
  xfs:xfs_fs_unfixed_corruption                      [Tracepoint event]
  xfs:xfs_fsmap_high_key                             [Tracepoint

  xfs:xfs_perag_set_cowblocks                        [Tracepoint event]
  xfs:xfs_perag_set_eofblocks                        [Tracepoint event]
  xfs:xfs_perag_set_reclaim                          [Tracepoint event]
  xfs:xfs_pwork_init                                 [Tracepoint event]
  xfs:xfs_read_agf                                   [Tracepoint event]
  xfs:xfs_read_agi                                   [Tracepoint event]
  xfs:xfs_read_extent                                [Tracepoint event]
  xfs:xfs_readdir                                    [Tracepoint event]
  xfs:xfs_readlink                                   [Tracepoint event]
  xfs:xfs_refcount_adjust_cow_error                  [Tracepoint event]
  xfs:xfs_refcount_adjust_error                      [Tracepoint event]
  xfs:xfs_refcount_cow_decrease                      [Tracepoint event]
  xfs:xfs_refcount_cow_increase                      [Tracepoint event]
  xfs:xfs_refcount_decrease                          [Tracepoint

  xfs:xfs_swap_extent_rmap_remap_piece               [Tracepoint event]
  xfs:xfs_symlink                                    [Tracepoint event]
  xfs:xfs_trans_add_item                             [Tracepoint event]
  xfs:xfs_trans_alloc                                [Tracepoint event]
  xfs:xfs_trans_bhold                                [Tracepoint event]
  xfs:xfs_trans_bhold_release                        [Tracepoint event]
  xfs:xfs_trans_binval                               [Tracepoint event]
  xfs:xfs_trans_bjoin                                [Tracepoint event]
  xfs:xfs_trans_brelse                               [Tracepoint event]
  xfs:xfs_trans_cancel                               [Tracepoint event]
  xfs:xfs_trans_commit                               [Tracepoint event]
  xfs:xfs_trans_commit_items                         [Tracepoint event]
  xfs:xfs_trans_dup                                  [Tracepoint event]
  xfs:xfs_trans_free                                 [Tracepoint

Memory_Lat:
  DRAM_Read_Latency
       [Average latency of data read request to external memory (in nanoseconds). Accounts for demand loads and L1/L2 prefetches]
  Load_Miss_Real_Latency
       [Actual Average Latency for L1 data-cache miss demand loads (in core cycles)]
No_group:
  L1MPKI
       [L1 cache true misses per kilo instruction for retired demand loads]
  L2HPKI_All
       [L2 cache hits per kilo instruction for all request types (including speculative)]
  L2MPKI
       [L2 cache true misses per kilo instruction for retired demand loads]
  L2MPKI_All
       [L2 cache misses per kilo instruction for all request types (including speculative)]
  L3MPKI
       [L3 cache true misses per kilo instruction for retired demand loads]
  Socket_CLKS
       [Socket actual clocks when any core is active on that socket]
PGO:
  BpTB
       [Branch instructions per taken branch]
  IFetch_Line_Utilization
       [Rough Estimation of fraction of fetched lines bytes that were likely (includes s

The above only lists the _named_ events. Typically there are hundreds more on the system. We will see how
to access those other events later in the tutorial. 

To get the number of loads and stores that go to the last-level cache (LLC) we can use the following
command with the `--event` option

In [19]:
perf stat --event LLC-loads,LLC-stores ./matvec 2000 200

522738635.7446

 Performance counter stats for './matvec 2000 200':

         3,118,067      LLC-loads                                                   
            88,376      LLC-stores                                                  

       2.379943299 seconds time elapsed

       2.367897000 seconds user
       0.011999000 seconds sys




**Do we have any new insight about the performance of matvec?**

### <a name="norm_perf"></a>Normalized performance

Execution time is not always a good measure of performance. Programs that execute more instructions
or those that process more data will have a longer execution time. That does not necessarily mean
that its performance is any worse that another program that executes fewer instructions. 

To get a better understanding of performance, we need a way to _normalize_ performance across
programs. One way to do this is to use a _throughput_ metrics. Throughput metrics measure
performance as a rate of _something_. For example, operating system performance may be measured in
number of tasks completed. FLOPs/sec counts the number of completed floating-operations per
second. This is the _de facto_ performance metric for HPC applications. This metric is also used to
rank the fastest supercomputers in the world by [top500.org](https://www.top500.org/). 

**How do we measure the FLOPS for `matvec`?**

We need to first find the event that corresponds to the execution of a floating-point
operation. This is not a named event. So we will need to dig up the hex-code. The code for the FP
event is `r538010`. We can now count the number of FP operations with perf

In [20]:
perf stat -e r538010 ./matvec 2000 200

522738635.7446

 Performance counter stats for './matvec 2000 200':

                 0      r538010                                                     

       2.379882309 seconds time elapsed

       2.367778000 seconds user
       0.012019000 seconds sys




`perf` will not give you the FLOPS directly but we can write a short script to calculate it.

In [24]:
# dump perf out to tmp file 
perf stat -e r538010 ./matvec 2000 200 2> tmp.prof
       
# extract flop count
flops=`cat tmp.prof | grep r538010  | awk '{print $1}' | sed 's/,//g'`
     
# extract number of seconds 
secs=`cat tmp.prof | grep "elapsed"  | awk '{print $1}'`
  
# calculate FLOPS
FLOPS=`echo $flops $secs | awk '{printf "%3.2f", ($1/1e+09)/$2}'`

522738635.7446
0


In [22]:
#print the result
echo "Performance = $FLOPS GFLOPS/s"

Performance = 0.00 GFLOPS/s
